In [1]:
install.packages("disk.frame", repo="https://cran.rstudio.com/")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
install.packages("MASS")
library(tidyverse)
library(caret)
library(dplyr)
library(glmnet)
library(reticulate)
#library(dplyr)
sagemaker <- import('sagemaker')
sagemaker <- import('sagemaker')
session <- sagemaker$Session()
bucket <- session$default_bucket()

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift

Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand

Loading required package: foreach

Attaching pac

In [3]:
# Load the data
triagedata <- get(load(file = "ScaledNumericData.RData"))
#triageData <- data
#sapply(triageData, class)
str(triagedata)
summary(triagedata)

'data.frame':	556298 obs. of  76 variables:
 $ ID                          : chr  "d5db547ae8e87d127ce0d4e51447befa5ff5012b_2019-08-04T02:50:00Z" "8ba1ffcd025f4002f0a94b9e5414a8868b40604f_2018-01-05T00:52:00Z" "75ccf97f6dbc795bd0bfc34067ab6b1a3d7e9325_2017-05-01T05:00:00Z" "ce3585171e5bf2dbaa366ebb9157cdbd790bcd66_2017-02-10T00:34:00Z" ...
 $ TRIAGE_CATEGORY             : num  3 3 2 3 3 3 3 2 4 3 ...
 $ AGE                         : num  0.331 0.29 0.537 0.824 0.331 ...
 $ ADMISSION_WITHIN_48_HOURS   : num  -0.26 -0.26 -0.26 -0.26 -0.26 ...
 $ AVPU_1                      : num  -0.1489 0.0287 0.0111 -0.1489 -0.1489 ...
 $ AVPU_1_DT_TM                : num  0.0307 -0.02648 -0.02648 -0.00368 0.01594 ...
 $ GCS_1                       : num  0.060655 0.000335 0.003378 0.256747 0.003378 ...
 $ GCS_1_DT_TM                 : num  -0.06299 -0.06299 -0.06299 -0.00181 -0.06299 ...
 $ RR_1                        : num  -0.4047 -0.2827 0.5204 -0.0963 -0.4047 ...
 $ RR_1_DT_TM                  : n

      ID            TRIAGE_CATEGORY      AGE          ADMISSION_WITHIN_48_HOURS
 Length:556298      Min.   :1.000   Min.   :-1.7236   Min.   :-0.2595          
 Class :character   1st Qu.:2.000   1st Qu.:-0.7373   1st Qu.:-0.2595          
 Mode  :character   Median :3.000   Median :-0.1208   Median :-0.2595          
                    Mean   :3.156   Mean   : 0.0000   Mean   : 0.0000          
                    3rd Qu.:4.000   3rd Qu.: 0.7832   3rd Qu.:-0.2595          
                    Max.   :5.000   Max.   : 2.6736   Max.   : 3.8528          
     AVPU_1           AVPU_1_DT_TM           GCS_1           
 Min.   :-0.148948   Min.   :-31.69604   Min.   :-22.484970  
 1st Qu.:-0.148948   1st Qu.: -0.02648   1st Qu.:  0.003378  
 Median :-0.148948   Median : -0.02461   Median :  0.143044  
 Mean   : 0.000000   Mean   :  0.00000   Mean   :  0.000000  
 3rd Qu.: 0.000915   3rd Qu.: -0.01004   3rd Qu.:  0.256747  
 Max.   :17.635976   Max.   :235.40949   Max.   :  0.256747  
  GCS_

In [4]:
#ED_data = select(data,-ID)
triagedata$ID <- NULL
ED_data <- triagedata

In [6]:
ED_data = triagedata %>% 
  #select(-ends_with('_2')) %>%
  select(-ends_with('1_DT_TM')) %>%
  #select(-ends_with('2_DT_TM')) %>%
  #select(-c(PRESENTING_PROBLEM, PATIENT_ID,TRIAGE_DT_TM,ID,AGE_GROUP)) %>%
  #select(-c(TRIAGE_PRESENTING_INFORMATION,TRIAGE_PRESENTING_ADDITIONAL_INFORMATION,PATIENT_PROBLEM_HISTORY)) %>%
  select(TRIAGE_CATEGORY, everything()) 
ED_data$TRIAGE_CATEGORY = as.numeric(ED_data$TRIAGE_CATEGORY)
str(ED_data)

'data.frame':	556298 obs. of  63 variables:
 $ TRIAGE_CATEGORY             : num  3 3 2 3 3 3 3 2 4 3 ...
 $ AGE                         : num  0.331 0.29 0.537 0.824 0.331 ...
 $ ADMISSION_WITHIN_48_HOURS   : num  -0.26 -0.26 -0.26 -0.26 -0.26 ...
 $ AVPU_1                      : num  -0.1489 0.0287 0.0111 -0.1489 -0.1489 ...
 $ GCS_1                       : num  0.060655 0.000335 0.003378 0.256747 0.003378 ...
 $ RR_1                        : num  -0.4047 -0.2827 0.5204 -0.0963 -0.4047 ...
 $ O2SATS_1                    : num  0.9333 -0.0826 -3.9544 -2.8682 0.3902 ...
 $ FIO2_1                      : num  -0.0553 -0.0448 -0.0497 0.9193 -0.0497 ...
 $ PULSE_1                     : num  0.258 -0.227 0.763 -0.292 -0.751 ...
 $ HEART_RATE_1                : num  -0.37 -0.377 -0.433 -0.415 -0.433 ...
 $ SBP_1                       : num  -0.145 0.242 -0.241 -0.145 1.298 ...
 $ DBP_1                       : num  0.604 0.567 0.604 -0.288 -0.824 ...
 $ MAP_1                       : num  0.15

In [7]:
names(ED_data)[names(ED_data) == "ENT, face"] <- "ENT_face"
names(ED_data)[names(ED_data) == "Mental Health"] <- "Mental_Health"
names(ED_data)[names(ED_data) == "Obstetrics and Gynaecology"] <- "Obstetrics_Gynaecology"

In [8]:
library(magrittr)
# Split the data into training and test set
set.seed(123)
training.samples <- ED_data$TRIAGE_CATEGORY %>%
  createDataPartition(p = 0.8, list = FALSE)
train.data  <- ED_data[training.samples, ]
test.data <- ED_data[-training.samples, ]
dim(train.data)


Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



[1] 445041     63

In [9]:
# Predictor variables
x <- model.matrix(TRIAGE_CATEGORY~ ., train.data)[,-1]
# Outcome variable
y <- train.data$TRIAGE_CATEGORY

In [10]:
glmnet(x, y, alpha = 1, lambda = NULL)


Call:  glmnet(x = x, y = y, alpha = 1, lambda = NULL) 

      Df    %Dev    Lambda
 [1,]  0 0.00000 0.3476000
 [2,]  1 0.02298 0.3167000
 [3,]  1 0.04206 0.2886000
 [4,]  1 0.05790 0.2629000
 [5,]  1 0.07105 0.2396000
 [6,]  1 0.08197 0.2183000
 [7,]  2 0.09798 0.1989000
 [8,]  2 0.11170 0.1812000
 [9,]  3 0.12590 0.1651000
[10,]  4 0.14030 0.1505000
[11,]  4 0.15640 0.1371000
[12,]  5 0.17420 0.1249000
[13,]  6 0.19180 0.1138000
[14,]  7 0.20770 0.1037000
[15,] 10 0.22510 0.0944900
[16,] 11 0.24240 0.0861000
[17,] 11 0.25700 0.0784500
[18,] 13 0.26950 0.0714800
[19,] 13 0.28020 0.0651300
[20,] 13 0.28900 0.0593400
[21,] 13 0.29630 0.0540700
[22,] 14 0.30240 0.0492700
[23,] 14 0.30770 0.0448900
[24,] 15 0.31210 0.0409000
[25,] 16 0.31620 0.0372700
[26,] 16 0.31960 0.0339600
[27,] 17 0.32250 0.0309400
[28,] 18 0.32500 0.0281900
[29,] 19 0.32730 0.0256900
[30,] 22 0.32960 0.0234100
[31,] 22 0.33170 0.0213300
[32,] 24 0.33350 0.0194300
[33,] 25 0.33510 0.0177100
[34,] 28 0.33670 0.016130

In [11]:
#Computing ridge regression
# Find the best lambda using cross-validation
set.seed(123) 
cv <- cv.glmnet(x, y, alpha = 0)
# Display the best lambda value
cv$lambda.min

[1] 0.03814643

In [12]:
# Fit the final model on the training data
model <- glmnet(x, y, alpha = 0, lambda = cv$lambda.min)
# Display regression coefficients
coef(model)

63 x 1 sparse Matrix of class "dgCMatrix"
                                        s0
(Intercept)                   3.155567e+00
AGE                          -1.650142e-01
ADMISSION_WITHIN_48_HOURS     1.676711e-02
AVPU_1                       -2.687100e-02
GCS_1                         4.980518e-02
RR_1                         -1.170464e-01
O2SATS_1                     -2.388349e-02
FIO2_1                       -3.130623e-04
PULSE_1                      -1.219474e-01
HEART_RATE_1                  1.397151e-02
SBP_1                        -5.871376e-02
DBP_1                         4.609453e-02
MAP_1                        -5.407700e-03
TEMP_TYMP_1                  -1.782308e-02
TEMP_ORAL_1                  -4.764100e-04
PATIENT_HEIGHT               -5.919232e-02
PATIENT_WEIGHT                2.253494e-04
PREGNANCY_STATUS             -4.819834e-02
SMOKING_STATUS               -1.285877e-02
RECENT_SURGERY_WITHIN_30_DAY  2.126074e-06
ANY_MEDICATION_USED          -4.184756e-03
PREVIOUS_ENC

In [13]:
#Make predictions on the test data
x.test <- model.matrix(TRIAGE_CATEGORY ~., test.data)[,-1]
predictions <- model %>% predict(x.test) %>% as.vector()
# Model performance metrics
data.frame(
  RMSE = RMSE(predictions, test.data$TRIAGE_CATEGORY),
  Rsquare = R2(predictions, test.data$TRIAGE_CATEGORY)
)

RMSE,Rsquare
0.7645369,0.3454458


In [14]:
#Computing lasso regression
# Find the best lambda using cross-validation
set.seed(123) 
cv <- cv.glmnet(x, y, alpha = 1)
# Display the best lambda value
cv$lambda.min

[1] 0.0003904405

In [15]:
# Fit the final model on the training data
model <- glmnet(x, y, alpha = 1, lambda = cv$lambda.min)
# Dsiplay regression coefficients
coef(model)

63 x 1 sparse Matrix of class "dgCMatrix"
                                        s0
(Intercept)                   3.155565e+00
AGE                          -1.773297e-01
ADMISSION_WITHIN_48_HOURS     1.479250e-02
AVPU_1                       -2.686227e-02
GCS_1                         5.232919e-02
RR_1                         -1.228898e-01
O2SATS_1                     -2.638789e-02
FIO2_1                        .           
PULSE_1                      -1.304692e-01
HEART_RATE_1                  6.041176e-03
SBP_1                        -6.064153e-02
DBP_1                         4.726861e-02
MAP_1                        -1.163862e-03
TEMP_TYMP_1                  -1.760102e-02
TEMP_ORAL_1                  -1.439481e-03
PATIENT_HEIGHT               -8.570050e-02
PATIENT_WEIGHT                1.562134e-02
PREGNANCY_STATUS             -4.769122e-02
SMOKING_STATUS               -1.272920e-02
RECENT_SURGERY_WITHIN_30_DAY  .           
ANY_MEDICATION_USED          -2.728185e-03
PREVIOUS_ENC

In [16]:
# Make predictions on the test data
x.test <- model.matrix(TRIAGE_CATEGORY ~., test.data)[,-1]
predictions <- model %>% predict(x.test) %>% as.vector()
# Model performance metrics
data.frame(
  RMSE = RMSE(predictions, test.data$TRIAGE_CATEGORY),
  Rsquare = R2(predictions, test.data$TRIAGE_CATEGORY)
)

RMSE,Rsquare
0.7642917,0.3455341


In [17]:
#Computing elastic net regession
# Build the model using the training set - ELASTICNET
set.seed(123)
model <- train(
  TRIAGE_CATEGORY ~., data = train.data, method = "glmnet",
  trControl = trainControl("cv", number = 10),
  tuneLength = 10
)
# Best tuning parameter
model$bestTune

,alpha,lambda
81,0.9,0.0001605891


In [18]:
# Coefficient of the final model. You need
# to specify the best lambda
coef(model$finalModel, model$bestTune$lambda)

63 x 1 sparse Matrix of class "dgCMatrix"
                                         1
(Intercept)                   3.155565e+00
AGE                          -1.774938e-01
ADMISSION_WITHIN_48_HOURS     1.482612e-02
AVPU_1                       -2.689213e-02
GCS_1                         5.230579e-02
RR_1                         -1.227664e-01
O2SATS_1                     -2.651902e-02
FIO2_1                        .           
PULSE_1                      -1.303034e-01
HEART_RATE_1                  5.710473e-03
SBP_1                        -6.063507e-02
DBP_1                         4.750507e-02
MAP_1                        -1.281590e-03
TEMP_TYMP_1                  -1.769104e-02
TEMP_ORAL_1                  -1.575827e-03
PATIENT_HEIGHT               -8.435181e-02
PATIENT_WEIGHT                1.420788e-02
PREGNANCY_STATUS             -4.772100e-02
SMOKING_STATUS               -1.279330e-02
RECENT_SURGERY_WITHIN_30_DAY -4.810549e-06
ANY_MEDICATION_USED          -2.805783e-03
PREVIOUS_ENC

In [19]:
# Make predictions on the test data
x.test <- model.matrix(TRIAGE_CATEGORY ~., test.data)[,-1]
predictions <- model %>% predict(x.test)
# Model performance metrics
data.frame(
  RMSE = RMSE(predictions, test.data$TRIAGE_CATEGORY),
  Rsquare = R2(predictions, test.data$TRIAGE_CATEGORY)
)

RMSE,Rsquare
0.7642905,0.3455363


In [20]:
#Comparing the different models
#Setup a grid range of lambda values:
lambda <- 10^seq(-3, 3, length = 100)

In [21]:
# Build the model Compute ridge regression:
set.seed(123)
ridge <- train(
  TRIAGE_CATEGORY ~., data = train.data, method = "glmnet",
  trControl = trainControl("cv", number = 10),
  tuneGrid = expand.grid(alpha = 0, lambda = lambda)
  )
# Model coefficients
coef(ridge$finalModel, ridge$bestTune$lambda)
# Make predictions
predictions <- ridge %>% predict(test.data)
# Model prediction performance
data.frame(
  RMSE = RMSE(predictions, test.data$TRIAGE_CATEGORY),
  Rsquare = R2(predictions, test.data$TRIAGE_CATEGORY)
)

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

63 x 1 sparse Matrix of class "dgCMatrix"
                                         1
(Intercept)                   3.155566e+00
AGE                          -1.657619e-01
ADMISSION_WITHIN_48_HOURS     1.661996e-02
AVPU_1                       -2.691383e-02
GCS_1                         4.988354e-02
RR_1                         -1.173975e-01
O2SATS_1                     -2.413697e-02
FIO2_1                       -2.932760e-04
PULSE_1                      -1.226789e-01
HEART_RATE_1                  1.480408e-02
SBP_1                        -5.886751e-02
DBP_1                         4.629692e-02
MAP_1                        -5.247566e-03
TEMP_TYMP_1                  -1.784129e-02
TEMP_ORAL_1                  -5.363779e-04
PATIENT_HEIGHT               -5.826499e-02
PATIENT_WEIGHT               -1.618105e-04
PREGNANCY_STATUS             -4.818090e-02
SMOKING_STATUS               -1.287959e-02
RECENT_SURGERY_WITHIN_30_DAY -2.511415e-05
ANY_MEDICATION_USED          -4.109642e-03
PREVIOUS_ENC

RMSE,Rsquare
0.7645074,0.3454384


In [22]:
# Build the model Compute lasso regression:
set.seed(123)
lasso <- train(
  TRIAGE_CATEGORY ~., data = train.data, method = "glmnet",
  trControl = trainControl("cv", number = 10),
  tuneGrid = expand.grid(alpha = 1, lambda = lambda)
  )
# Model coefficients
coef(lasso$finalModel, lasso$bestTune$lambda)
# Make predictions
predictions <- lasso %>% predict(test.data)
# Model prediction performance
data.frame(
  RMSE = RMSE(predictions, test.data$TRIAGE_CATEGORY),
  Rsquare = R2(predictions, test.data$TRIAGE_CATEGORY)
)

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

63 x 1 sparse Matrix of class "dgCMatrix"
                                         1
(Intercept)                   3.155564e+00
AGE                          -1.764519e-01
ADMISSION_WITHIN_48_HOURS     1.433039e-02
AVPU_1                       -2.630358e-02
GCS_1                         5.504711e-02
RR_1                         -1.223040e-01
O2SATS_1                     -2.541577e-02
FIO2_1                        .           
PULSE_1                      -1.288827e-01
HEART_RATE_1                  1.929791e-03
SBP_1                        -6.016980e-02
DBP_1                         4.614889e-02
MAP_1                         .           
TEMP_TYMP_1                  -1.713621e-02
TEMP_ORAL_1                  -6.029299e-04
PATIENT_HEIGHT               -7.295456e-02
PATIENT_WEIGHT                3.025561e-05
PREGNANCY_STATUS             -4.739158e-02
SMOKING_STATUS               -1.208661e-02
RECENT_SURGERY_WITHIN_30_DAY  .           
ANY_MEDICATION_USED          -1.934746e-03
PREVIOUS_ENC

RMSE,Rsquare
0.7643374,0.345461


In [23]:
# Build the model Elastic net regression
set.seed(123)
elastic <- train(
  TRIAGE_CATEGORY ~., data = train.data, method = "glmnet",
  trControl = trainControl("cv", number = 10),
  tuneLength = 10
  )
# Model coefficients
coef(elastic$finalModel, elastic$bestTune$lambda)
# Make predictions
predictions <- elastic %>% predict(test.data)
# Model prediction performance
data.frame(
  RMSE = RMSE(predictions, test.data$TRIAGE_CATEGORY),
  Rsquare = R2(predictions, test.data$TRIAGE_CATEGORY)
)

63 x 1 sparse Matrix of class "dgCMatrix"
                                         1
(Intercept)                   3.155565e+00
AGE                          -1.774938e-01
ADMISSION_WITHIN_48_HOURS     1.482612e-02
AVPU_1                       -2.689213e-02
GCS_1                         5.230579e-02
RR_1                         -1.227664e-01
O2SATS_1                     -2.651902e-02
FIO2_1                        .           
PULSE_1                      -1.303034e-01
HEART_RATE_1                  5.710473e-03
SBP_1                        -6.063507e-02
DBP_1                         4.750507e-02
MAP_1                        -1.281590e-03
TEMP_TYMP_1                  -1.769104e-02
TEMP_ORAL_1                  -1.575827e-03
PATIENT_HEIGHT               -8.435181e-02
PATIENT_WEIGHT                1.420788e-02
PREGNANCY_STATUS             -4.772100e-02
SMOKING_STATUS               -1.279330e-02
RECENT_SURGERY_WITHIN_30_DAY -4.810549e-06
ANY_MEDICATION_USED          -2.805783e-03
PREVIOUS_ENC

RMSE,Rsquare
0.7642905,0.3455363


In [24]:
models <- list(ridge = ridge, lasso = lasso, elastic = elastic)
resamples(models) %>% summary( metric = "RMSE")


Call:
summary.resamples(object = ., metric = "RMSE")

Models: ridge, lasso, elastic 
Number of resamples: 10 

RMSE 
             Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ridge   0.7601141 0.7625952 0.7651855 0.7641747 0.7659025 0.7672575    0
lasso   0.7598294 0.7623348 0.7649737 0.7639483 0.7656544 0.7671686    0
elastic 0.7597656 0.7622812 0.7649383 0.7638989 0.7656334 0.7671351    0
